In [36]:
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer,WordNetLemmatizer
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from datasets import load_dataset
import unicodedata

In [41]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
raw_dataset = load_dataset("conv_ai_2")

Reusing dataset conv_ai_2 (C:\Users\justi\.cache\huggingface\datasets\conv_ai_2\conv_ai_2\1.0.0\11d600ddce66bb9d07ca50d1b55b488145ef0d5d0206168c32f1043677875865)


In [7]:
raw_train_dataset = raw_dataset["train"]
raw_dialog_dataset = raw_train_dataset["dialog"]

In [15]:
def get_dialog(dataset):
    dataset_length = len(dataset)
    full_dialog = []
    for i in range(dataset_length - 1000):
        conv_length = len(dataset[i])
        for j in range(conv_length):
            inputLine = dataset[i][j]['text'].strip() 
            if not isinstance(inputLine, str): 
                inputLine = 'None'
            full_dialog.append(inputLine)
    return full_dialog

In [30]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

In [31]:
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

In [32]:
def stem(text):
    ps = nltk.stem.porter.PorterStemmer()
    new_text = [ps.stem(word) for word in text]
    return new_text

In [33]:
def lemmatize(text):
    lem = nltk.stem.wordnet.WordNetLemmatizer()
    new_text = [lem.lemmatize(word) for word in text]
    return new_text

In [58]:
def response(user_response):
    full_dialog = get_dialog(raw_dialog_dataset)
    normalized_dialog = [normalizeString(s) for s in full_dialog]
    stemmed_dialog = stem(normalized_dialog)
    lem_dialog = lemmatize(stemmed_dialog)
    
    chatbot_resp=""
    full_dialog.append(user_response)

    TfidVec=TfidfVectorizer()
    tfidf=TfidVec.fit_transform(lem_dialog)
    vals=cosine_similarity(tfidf[-1],tfidf)
    idx=vals.argsort()[0][-2]
    flat=vals.flatten()
    flat.sort()
    req_tfidf=flat[-2]
    if(req_tfidf==0):
        chatbot_resp+="Sorry!"
        return chatbot_resp
    else:
        chatbot_resp+=lem_dialog[idx]
        return chatbot_resp


In [59]:
res = response("I am a vegan")
print("Chatbot Response : = ",res)

Chatbot Response : =  hello


In [17]:
full_dialog = get_dialog(raw_dialog_dataset)

In [37]:
normalized_dialog = [normalizeString(s) for s in full_dialog]

In [38]:
stemmed_dialog = stem(normalized_dialog)

In [42]:
lem_dialog = lemmatize(stemmed_dialog)

In [43]:
lem_dialog

['i love iphone ! i just bought new iphone !',
 'thats good for you i m not very into new tech',
 'i am a college student and i am a college stud',
 'i am go to gym and live on don',
 'i am a vegan and i am in the midwest',
 'so vegan . . . i have dogs maybe i should told then that they may eat cheap salads insted of meat',
 'i would not mind having them in the office that would be hard for m',
 'dogs or vegan in office ?',
 'i am a vegetarian so i am vegan',
 'strange answ',
 'i guess i just do not know what to ps experience with .',
 'hello',
 'test',
 'hey',
 'what do you do ?',
 'i love painting i love to paint',
 'hello',
 'start',
 'hello',
 'municipal junior high school entrance ceremony the next year i left home to live in the dorms at the junior high school .',
 'hello',
 'hi',
 'that sounds nice . i love green .',
 'llo',
 'hello',
 'hello how are you ?',
 'what is your name ?',
 'hello',
 'anybody ther',
 'on the ball city the oldest football song in the world .',
 'what s u